In [1]:
import pandas as pd
import numpy as np
import os
import subprocess 
import glob

from basic_tools import *

http://www.nealelab.is/uk-biobank

In [5]:
ukbb_table=pd.read_csv(ukbb_table_path,sep=',')

In [6]:
ukbb_table.head()

,Phenotype Code,check,Phenotype Description,UK Biobank Data Showcase Link,Sex,File,wget command,Dropbox File
0,NaN,1,README,NaN,NaN,README,wget https://www.dropbox.com/s/wro30igqkmit5ig...,https://www.dropbox.com/s/wro30igqkmit5ig/READ...
1,NaN,1,List of European samples,NaN,both_sexes,european_samples.tsv.bgz,wget https://www.dropbox.com/s/bvnd723tl67lh8v...,https://www.dropbox.com/s/bvnd723tl67lh8v/euro...
2,NaN,1,List of samples used in GWAS - both_sexes,NaN,both_sexes,samples.both_sexes.tsv.bgz,wget https://www.dropbox.com/s/ypcmzukh2vwhtkh...,https://www.dropbox.com/s/ypcmzukh2vwhtkh/samp...
3,NaN,1,List of samples used in GWAS - female,NaN,female,samples.female.tsv.bgz,wget https://www.dropbox.com/s/nnn2zc3z5uqmlra...,https://www.dropbox.com/s/nnn2zc3z5uqmlra/samp...
4,NaN,1,List of samples used in GWAS - male,NaN,male,samples.male.tsv.bgz,wget https://www.dropbox.com/s/r2o91gr3r4l9kz9...,https://www.dropbox.com/s/r2o91gr3r4l9kz9/samp...


In [7]:
phenotypes_both_sexes_v2=pd.read_csv(phenotypes_both_sexes_v2_file_path,sep='\t',compression='gzip')
phenotypes_male_v2=pd.read_csv(phenotypes_male_v2_file_path,sep='\t',compression='gzip')
phenotypes_female_v2=pd.read_csv(phenotypes_female_v2_file_path,sep='\t',compression='gzip')
phenotypes_both_sexes_v2['Sex']='both_sexes';phenotypes_male_v2['Sex']='male';phenotypes_female_v2['Sex']='female';
phenotypes=pd.concat([phenotypes_both_sexes_v2,phenotypes_male_v2,phenotypes_female_v2],sort='False')

In [ ]:
phenotypes_select_list=[]
for idx,phenotype in enumerate(phenotypes['phenotype'].unique()):
    ukbb_table_select=ukbb_table[ukbb_table['Phenotype Code']==phenotype]
    if (ukbb_table_select['check']==0).sum()>0:
        continue
    phenotypes_select=phenotypes[phenotypes['phenotype']==phenotype]
    if phenotypes_select[phenotypes_select['Sex']=='female'].shape[0]==0:
        phenotypes_select_list.append(phenotypes_select[phenotypes_select['Sex']=='male'])
        print(phenotypes_select[phenotypes_select['Sex']=='male'][['Sex','description']])
    elif phenotypes_select[phenotypes_select['Sex']=='male'].shape[0]==0:
        phenotypes_select_list.append(phenotypes_select[phenotypes_select['Sex']=='female'])
        print(phenotypes_select[phenotypes_select['Sex']=='female'][['Sex','description']])
    else:
        phenotypes_select_list.append(phenotypes_select[phenotypes_select['Sex']=='both_sexes'])
phenotypes_filtered=pd.concat(phenotypes_select_list)

In [2]:
phenotypes_filtered.shape

NameError: name 'phenotypes_filtered' is not defined

In [10]:
len(ukbb_table['Phenotype Code'][ukbb_table['check']!=0].unique())

3486

In [11]:
ukbb_table_file_list=[]
for idx,row in phenotypes_filtered.iterrows():
    ukbb_table_select=ukbb_table[ukbb_table['Phenotype Code']==row['phenotype']]
    ukbb_table_select=ukbb_table_select[ukbb_table_select['Sex']==row['Sex']]
    
    if ukbb_table_select['File'].str.contains('v2').sum()>0:
        ukbb_table_select=ukbb_table_select[ukbb_table_select['File'].str.contains('v2')]
    #& (ukbb_table['Sex']==row['Sex'])]
    #print(ukbb_table_select)
    if ukbb_table_select.shape[0]==1:
        ukbb_table_file_list.append(ukbb_table_select.iloc[0][['Phenotype Code','UK Biobank Data Showcase Link','File','wget command','Dropbox File']])
    else:
        print(ukbb_table_select)
        

In [12]:
phenotypes_filtered=phenotypes_filtered.merge(right=pd.DataFrame(ukbb_table_file_list),left_on='phenotype',right_on='Phenotype Code')

In [13]:
phenotypes_filtered=phenotypes_filtered.set_index('phenotype')

In [14]:
phenotypes_filtered.to_csv(phenotypes_filtered_path)

In [84]:
os.chdir(gwas_path)

In [ ]:
for idx,row in phenotypes_filtered.iterrows():
    if not os.path.exists(row['File']):
        run_command(row['wget command'],quiet=True)